In [43]:
import json

path=r"C:\Fall 2022 courses\Project\Data engineer project\AWS Glue ETL\convertcsv.json"
df=pd.read_json(path)

In [44]:
df=df.drop(columns=['Cases - cumulative total per 100000 population', 'Cases - newly reported in last 7 days per 100000 population','Deaths - cumulative total per 100000 population','Deaths - newly reported in last 7 days per 100000 population', 'FIELD13'  ])

In [45]:
df

,Name,WHO Region,Cases - cumulative total,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours
0,Global,,660378145,2948506,230705,6691495,10848,937
1,United States of America,Americas,99888608,462944,0,1084932,2695,0
2,India,South-East Asia,44680386,1255,171,530722,15,0
3,France,Europe,38323809,62779,11715,159453,724,115
4,Germany,Europe,37562191,102646,16368,163244,112,10
...,...,...,...,...,...,...,...,...
233,Holy See,Europe,26,0,0,0,0,0
234,Tokelau,Western Pacific,5,0,0,0,0,0
235,Pitcairn Islands,Western Pacific,4,0,0,0,0,0
236,Democratic People's Republic of Korea,South-East Asia,0,0,0,0,0,0


In [ ]:
import json
import psycopg2
import pandas as pd
import os

def lambda_handler(event, context):
    print("event collected is {}".format(event))
    for record in event['Records'] :
        s3_bucket = record['s3']['bucket']['name']
        print("Bucket name is {}".format(s3_bucket))
        s3_key = record['s3']['object']['key']
        print("Bucket key name is {}".format(s3_key))
        path = "s3://{}/{}".format(s3_bucket, s3_key)
        print("from path {}".format(path))
        
        df=pd.read_json(path)
        df=df.drop(columns=['Cases - cumulative total per 100000 population', 'Cases - newly reported in last 7 days per 100000 population','Deaths - cumulative total per 100000 population','Deaths - newly reported in last 7 days per 100000 population', 'FIELD13'])
        
        Access_key = os.getenv('AWS_Access_key')
        Access_Secrete = os.getenv('AWS_Access_Secrete')
        dbname = os.getenv('redshift-cluster-1')
        host = os.getenv('host')
        user = os.getenv('user')
        password = os.getenv('Vitthal@1234')
        tablename = os.getenv('covid_analysis')
        connection = psycopg2.connect(dbname = dbname,
                                       host = host,
                                       port = '5439',
                                       user = user,
                                       password = password)
                                       
        print('after connection....')
        cur = connection.cursor()
        print('after cursor....')
        
        covid_data_table_insert=("""INSERT INTO covid_analysis(
        name,
        who_region,
        cases_cumulative_total,
        cases_newly_reported_in_last_7_days,
        cases_newly_reported_in_last_24_hours,
        deaths_cumulative_total,
        deaths_newly_reported_in_last_7_days,
        deaths_newly_reported_in_last_24_hours
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """)
        
        for j, row in df.iterrows():
            cur.execute(covid_data_table_insert, list(row))
            
        connection.commit()

        print('after execute....')
        cur.close()
        print('after curs close....')
        connection.close()
        print('after connection close....')